## Труд всем ру

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
from time import sleep
import math
from bs4 import BeautifulSoup
import os
import requests
import re
import xlsxwriter

In [91]:



def get_vacation_urls(vacantion_search_url=r"https://trudvsem.ru/vacancy/search", num_pages=0):
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    # options.add_argument("--headless")

    #options.add_experimental_option("excludeSwitches", ["enable-automation"])
    #options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(options=options)  # 

    stealth(driver,
            languages=["ru", "ru-RU"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    driver.get(vacantion_search_url)
    start_button = driver.find_element(By.XPATH, "//button[@class='search-content__button']")
    start_button.click()
    sleep(3)
    if num_pages == 0:
        num_vacancy_text = driver.find_element(By.CLASS_NAME, 'ib-filter__result-counter').text
        num_vacancy = int(num_vacancy_text[:num_vacancy_text.find(' ')])
        print(num_vacancy_text)
        for _ in range(math.ceil(num_vacancy / 10)):
            sleep(1)
            element = driver.find_elements(By.CLASS_NAME, 'button_secondary')
            for e in element:
                if e.text == 'Загрузить ещё':
                    driver.execute_script("arguments[0].click();", e)
        for i in range(num_vacancy):
            elem = driver.find_element(By.XPATH, '//div[@class="search-results-simple-card mb-1"]')
            info_div = elem.find_elements(By.XPATH,
                                          '//div[@class="search-results-simple-card__wrapper search-results-simple-card__wrapper_column"]')

            soup_vacancy_info = BeautifulSoup(info_div[i].get_attribute('innerHTML'), 'lxml')
            employer, region = list(
                map(lambda x: x.text, soup_vacancy_info.find_all('div', {'class': 'content_small content_clip'})))

            driver.execute_script("arguments[0].click();", elem)
            soup = BeautifulSoup(driver.page_source, "lxml")
            vacancy_name_html = soup.find('a', {'class': "link link_title"})
            if not vacancy_name_html:
                while not vacancy_name_html:
                    soup = BeautifulSoup(driver.page_source, "lxml")

            vacancy_name = vacancy_name_html.text
            print(vacancy_name)
            salary = soup.find('span',
                               {'class': 'content__section-subtitle search-results-full-card__salary'}).text.strip()
            print(salary)
            date = soup.find('span', {'class': 'content_small content_pale'}).text
            date = date[date.find(' ') + 1:]
            print(date)
            print(region)
            print(employer)
            vacancy_descr = soup.find('div',
                                      {'class': "tabs__content tabs_active", 'id': "vacancy-details"}).text.split()
            vacancy_descr = ' '.join(list(map(lambda x: x.strip(), vacancy_descr)))
            print(vacancy_descr)
            requirements = vacancy_descr[
                           vacancy_descr.find('Требования к кандидату'):vacancy_descr.find('Данные по вакансии')]
            print('requirements:')
            print(requirements)
            print('\n')
            if 'Опыт работы' in requirements:
                print('experience:')
                experience = requirements[requirements.find('Опыт работы'):]
                print(experience[:[m.start() for m in re.finditer(' ', experience)][4]])
                #print(experience)
            else:
                experience = ''
                print(experience)

            if 'График' in requirements:
                print('schedule:')
                schedule = requirements[requirements.find('График'):]
                print(schedule)
                #print(schedule[:[m.start() for m in re.finditer(' ', schedule)][1]])
                #print(experience)
            else:
                schedule = ''
                print(schedule)

            print(i)

    # elif num_pages >= 1:
    #     for _ in range(num_pages):
    #         sleep(1)
    #         element = driver.find_elements(By.CLASS_NAME, 'button_secondary')
    #         for e in element:
    #             if e.text == 'Загрузить ещё':
    #                 driver.execute_script("arguments[0].click();", e)


get_vacation_urls(
    'https://trudvsem.ru/vacancy/search?_title=%D1%8D%D0%BA%D1%81%D0%BA%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%BE%D0%B4')

132 вакансии
Экскурсовод
от 23 400 руб.
14.03.2024
Республика Башкортостан
ИП Валекжанин Д.Ю.
Должностные обязанности Организует и проводит экскурсии, туристические походы с закрепленными за ним туристскими группами. Временно, ненормированный рабочий день, график. Желательно знание иностранных языков. Опыт работы от 3 лет. Требования к кандидату Временно, ненормированный рабочий день, график. Желательно знание иностранных языков, в том числе филиппинский. Опыт работы от 3 лет. Опыт работы от 3 лет Образование: Среднее профессиональное Данные по вакансии Профессия: Экскурсовод График работы: Ненормированный рабочий день Количество рабочих мест: 1 Премии и бонусы Наличие общежития Предоставление жилья при переезде Соцпакет Предоставление жилья при переезде Тип жилья: Общежитие Вакансия на карте Информация о регионе Детские сады Детские кружки Школы и техникумы ВУЗы Медицина Культура и досуг Спорт Выбранные объекты отображаются только на масштабе менее 2 км …© Яндекс Условия использования

KeyboardInterrupt: 

In [10]:

vacantion_search_url = 'https://trudvsem.ru/vacancy/search'
num_pages = 0
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

book = xlsxwriter.Workbook(r"./data/parced_data.xlsx")
page = book.add_worksheet("данные")

page.set_column("A:A", 20)
page.set_column("B:B", 50)
page.set_column("C:C", 20)
page.set_column("D:D", 20)
page.set_column("E:E", 20)
page.set_column("F:F", 20)
page.set_column("G:G", 20)
page.set_column("H:H", 20)
page.set_column("I:I", 20)
page.set_column("J:J", 20)


# options.add_argument("--headless")

#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)  # 

stealth(driver,
        languages=["ru", "ru-RU"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get(vacantion_search_url)
start_button = driver.find_element(By.XPATH, "//button[@class='search-content__button']")
start_button.click()
sleep(3)
if num_pages == 0:
    try:
        num_vacancy_text = driver.find_element(By.CLASS_NAME, 'ib-filter__result-counter').text
        num_vacancy = int(''.join(num_vacancy_text[:num_vacancy_text.rfind(' ')].split()))
        print(num_vacancy_text)
        for _ in range(math.ceil(num_vacancy / 10)):
            sleep(1)
            element = driver.find_elements(By.CLASS_NAME, 'button_secondary')
            for e in element:
                if e.text == 'Загрузить ещё':
                    driver.execute_script("arguments[0].click();", e)
        for i in range(num_vacancy):
            elem = driver.find_element(By.XPATH, '//div[@class="search-results-simple-card mb-1"]')
            info_div = elem.find_elements(By.XPATH,
                                          '//div[@class="search-results-simple-card__wrapper search-results-simple-card__wrapper_column"]')
    
            soup_vacancy_info = BeautifulSoup(info_div[i].get_attribute('innerHTML'), 'lxml')
            employer, region = list(
                map(lambda x: x.text, soup_vacancy_info.find_all('div', {'class': 'content_small content_clip'})))
    
            driver.execute_script("arguments[0].click();", elem)
            soup = BeautifulSoup(driver.page_source, "lxml")
            vacancy_name_html = soup.find('a', {'class': "link link_title"})
            if not vacancy_name_html:
                while not vacancy_name_html:
                    soup = BeautifulSoup(driver.page_source, "lxml")
    
            vacancy_name = vacancy_name_html.text
            print(vacancy_name)
            salary = soup.find('span', {'class': 'content__section-subtitle search-results-full-card__salary'}).text.strip()
            print(salary)
            date = soup.find('span', {'class': 'content_small content_pale'}).text
            date = date[date.find(' ') + 1:]
            print(date)
            print(region)
            print(employer)
            vacancy_descr = soup.find('div', {'class': "tabs__content tabs_active", 'id': "vacancy-details"}).text.split()
            vacancy_descr = ' '.join(list(map(lambda x: x.strip(), vacancy_descr)))
            print(vacancy_descr)
            requirements = vacancy_descr[
                           vacancy_descr.find('Требования к кандидату'):vacancy_descr.find('Данные по вакансии')]
            print('requirements:')
            print(requirements)
            print('\n')
            if 'Опыт работы' in requirements:
                print('experience:')
                experience = requirements[requirements.find('Опыт работы'):]
                experience = experience[:[m.start() for m in re.finditer(' ', experience)][4]]
                print(experience)
            else:
                experience = ''
                print(experience)
    
            if 'График работы' in vacancy_descr:
                print('schedule:')
                schedule = vacancy_descr[vacancy_descr.find('График работы'):]
                schedule = schedule[:[m.start() for m in re.finditer(' ', schedule)][2]]
                print(schedule)
            else:
                schedule = ''
                print(schedule)
    
            data = [vacancy_name, vacancy_descr, salary, date, region, employer, requirements, experience, schedule]
            column_num = 0
            for col in data:
                page.write(i, column_num, col)
                column_num += 1
    
            print(i)
    except:
        print('Завершено')
    book.close()

68 040 вакансий
Завершено


In [9]:
int(''.join('68 000'.split()))

68000

In [ ]:
import xlsxwriter


def writer(parametr):
    book = xlsxwriter.Workbook(r"C:\Users\igorv\OneDrive\Documents\GitHub\Parsers\already_parse\data.xlsx")
    page = book.add_worksheet("товар")

    row = 0
    column = 0

    page.set_column("A:A", 20)
    page.set_column("B:B", 20)
    page.set_column("C:C", 50)
    page.set_column("D:D", 50)

    for item in parametr():
        page.write(row, column, item[0])
        page.write(row, column + 1, item[1])
        page.write(row, column + 2, item[2])
        page.write(row, column + 3, item[3])
        row += 1

    book.close()


writer(array)

In [99]:
requirements

'Требования к кандидату Дисциплинорованность, ответственность, коммуникабельность Квалификация: высшее профессиональное образование, дополнительную подготовку, стаж организаторской работы в сфере туризма и путешествий не менее одного года. Образование: Высшее Специальность по образованию: Туризм '

## Работа ру

In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
import time

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

# options.add_argument("--headless")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Users\DIPRAJ\Programming\adclick_bot\chromedriver.exe")

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

url = "https://bot.sannysoft.com/"
driver.get(url)
time.sleep(5)
driver.quit()

In [23]:
def get_page_data(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.0.3) Gecko/20091102 Firefox/3.5.5 (.NET CLR 3.5.30729)"}
    main_url = r"https://trudvsem.ru"
    response = requests.get(main_url + url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    print(soup)
    # vacation_name = soup.find('div',{'class': "col-12 col-md-6 mb-2 mb-md-0"})
    # print(vacation_name)


for article in all_publications[:2]:
    get_page_data(article['href'])

<!DOCTYPE html>
<html lang="ru">
<head data-path="/assets/redesign-theme/uikit/?r=1711340465847">
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Вакансия Экскурсовод - ИП Валекжанин Д.Ю., работа в Республика Башкортостан</title>
<meta content="Вакансия Экскурсовод. Зарплата от 23400 до 0. В ИП Валекжанин Д.Ю. в Республика Башкортостан. График работы Ненормированный рабочий день. Опубликовано 14.03.2024." name="description"/>
<link href="https://trudvsem.ru/vacancy/card/312028015800042/31c000c2-5f81-11ed-a1ff-93f7cc862eb6" rel="canonical"/>
<meta content="https://trudvsem.ru/vacancy/card/312028015800042/31c000c2-5f81-11ed-a1ff-93f7cc862eb6" property="og:url"/>
<meta content="Работа России - Общероссийская база вакансий и резюме. Государственный портал trudvsem.ru" property="og:site_name"/>
<meta content="Вакансия Экскурсовод - ИП Валекжанин Д.Ю., работа в Респуб

In [ ]:
button
button_icon
button_plain
ml - 1
search - results - simple - card__go - to - button

In [17]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.0.3) Gecko/20091102 Firefox/3.5.5 (.NET CLR 3.5.30729)"}
list_card_url = []

url = f"https://scrapingclub.com/exercise/list_basic/?page={count}"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")  #html.parser
data = soup.find_all("div", class_="col-lg-4 col-md-6 mb-4")

In [22]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.0.3) Gecko/20091102 Firefox/3.5.5 (.NET CLR 3.5.30729)"}
list_card_url = []


def get_url():
    for count in range(1, 8):
        url = f"https://scrapingclub.com/exercise/list_basic/?page={count}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "lxml")  #html.parser
        data = soup.find_all("div", class_="col-lg-4 col-md-6 mb-4")
        for i in data:
            card_url = "https://scrapingclub.com" + i.find("a").get("href")
            yield card_url


def array():
    for card_url in get_url():
        response = requests.get(card_url, headers=headers)
        sleep(3)
        soup = BeautifulSoup(response.text, "lxml")
        data = soup.find("div", class_="card mt-4 my-4")
        name = data.find("h3", class_="card-title").text.replace("\n", "")
        price = data.find("h4").text
        text = data.find("p", class_="card-text").text
        url_img = "https://scrapingclub.com" + data.find("img", class_="card-img-top img-fluid").get("src")
        yield name, price, text, url_img

Short Dress 
 $24.99 
 Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold. 
 https://scrapingclub.com/static/img/90008-E.jpg
Patterned Slacks 
 $29.99 
 Ankle-length slacks in patterned stretch cotton satin. Regular waist with concealed hook-and-eye fastener and zip fly. Side pockets and tapered legs with slits at hems. 61% cotton, 36% polyester, 3% spandex. Machine wash... 
 https://scrapingclub.com/static/img/96436-A.jpg
Short Chiffon Dress 
 $49.99 
 Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm. 
 https://scrapingclub.com/static/img/93926-B.jpg
Off-the-shoulder Dress 
 $59.99 
 Short, fitted off-the-shoulder dress in 

In [14]:
# name = i.find("h4", class_="card-title").text.replace("\n", "")
# price = i.find("h5").text
# url_img = "https://scrapingclub.com"+i.find("img", class_="card-img-top img-fluid").get("src")
# print(name, '\n', price, '\n', url_img)

Short Dress 
 $24.99 
 https://scrapingclub.com/static/img/90008-E.jpg
Patterned Slacks 
 $29.99 
 https://scrapingclub.com/static/img/96436-A.jpg
Short Chiffon Dress 
 $49.99 
 https://scrapingclub.com/static/img/93926-B.jpg
Off-the-shoulder Dress 
 $59.99 
 https://scrapingclub.com/static/img/90882-B.jpg
V-neck Top 
 $24.99 
 https://scrapingclub.com/static/img/93756-C.jpg
Short Chiffon Dress 
 $49.99 
 https://scrapingclub.com/static/img/93926-C.jpg
V-neck Top 
 $24.99 
 https://scrapingclub.com/static/img/93756-B.jpg
V-neck Top 
 $24.99 
 https://scrapingclub.com/static/img/93756-D.jpg
Short Lace Dress 
 $59.99 
 https://scrapingclub.com/static/img/96643-A.jpg


In [24]:
import xlsxwriter
from parser_file import array


def writer(parametr):
    book = xlsxwriter.Workbook(r"C:\Users\igorv\OneDrive\Documents\GitHub\Parsers\already_parse\data.xlsx")
    page = book.add_worksheet("товар")

    row = 0
    column = 0

    page.set_column("A:A", 20)
    page.set_column("B:B", 20)
    page.set_column("C:C", 50)
    page.set_column("D:D", 50)

    for item in parametr():
        page.write(row, column, item[0])
        page.write(row, column + 1, item[1])
        page.write(row, column + 2, item[2])
        page.write(row, column + 3, item[3])
        row += 1

    book.close()


writer(array)

parsing file - parser_file.py, save parsing_data to excel - in_excel.